# SET DESIRED EVALUATION PARAMS HERE

In [ ]:
SPLIT = 'dev' # 'dev' or 'test'
TEMPERATURE = '1' # '0', '0.1' or '1'
MODEL = 'neox' # 'flan-t5', 'neox' or 'davinci'
ONE_SHOT = False # True or False

# Imports

In [ ]:
import re
import regex
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from Levenshtein import distance
from collections import Counter

# Constants

In [ ]:
COLUMN_ORDER = ["raw", "Address (post town)", "Address (post code)", "Address (street)", "Charity Name", "Charity Number", "Annual Income", "Period End Date", "Annual Spending"]

In [ ]:
if SPLIT == 'dev':
    if MODEL == 'flan-t5':
        if ONE_SHOT:
            if TEMPERATURE == '0':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T23-40-17_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T23-40-19_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T23-40-22_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=True).tsv',
                ]
                LOG_PATHS = [
                    '../logs/flan-t5/2023-01-19T23-40-17_flan-t5_temp_0.log',
                    '../logs/flan-t5/2023-01-19T23-40-19_flan-t5_temp_0.log',
                    '../logs/flan-t5/2023-01-19T23-40-22_flan-t5_temp_0.log',
                ]
        else:
            if TEMPERATURE == '0':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T15-01-08_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T15-01-09_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T15-01-10_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                ]
                LOG_PATHS = [
                    '../logs/flan-t5/2023-01-19T15-01-08_flan-t5_temp_0.log',
                    '../logs/flan-t5/2023-01-19T15-01-09_flan-t5_temp_0.log',
                    '../logs/flan-t5/2023-01-19T15-01-10_flan-t5_temp_0.log',
                ]

            elif TEMPERATURE == '0.1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T15-02-03_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T22-18-34_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T22-22-59_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                ]
                LOG_PATHS = [
                    '../logs/flan-t5/2023-01-19T15-02-03_flan-t5_temp_0.1.log',
                    '../logs/flan-t5/2023-01-19T22-18-34_flan-t5_temp_0.1.log',
                    '../logs/flan-t5/2023-01-19T22-22-59_flan-t5_temp_0.1.log',
                ]

            elif TEMPERATURE == '1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T17-41-36_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T17-51-37_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                    'datasets/kleister_charity/dev-0/predictions/flan-t5/2023-01-19T22-13-13_LLMPipeline(prompt_variant=NeutralPrompt, model=Flan_T5(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40), parser=KleisterCharityParser, shots=False).tsv',
                ]
                LOG_PATHS = [
                    '../logs/flan-t5/2023-01-19T17-41-36_flan-t5_temp_1.log',
                    '../logs/flan-t5/2023-01-19T17-51-37_flan-t5_temp_1.log',
                    '../logs/flan-t5/2023-01-19T22-13-13_flan-t5_temp_1.log',
                ]
                
    elif MODEL == 'neox':
        if ONE_SHOT:
            if TEMPERATURE == '1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/neox/2023-01-15T23-35-37_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2023-01-15T23-44-53_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2023-01-15T23-45-00_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                ]
                LOG_PATHS = [
                    '../logs/neox/2023-01-15T23-35-37_neox_oneshot_temp_1.log',
                    '../logs/neox/2023-01-15T23-44-53_neox_oneshot_temp_1.log',
                    '../logs/neox/2023-01-15T23-45-00_neox_oneshot_temp_1.log',
                ]

        else:
            if TEMPERATURE == '0':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-14T18-27-21_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-15T21-44-54_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-15T21-45-15_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/neox/2022-12-14T18-27-21_neox_temp_0.log',
                    '../logs/neox/2022-12-15T21-44-54_neox_temp_0.log',
                    '../logs/neox/2022-12-15T21-45-15_neox_temp_0.log',
                ]

            elif TEMPERATURE == '0.1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-14T18-27-46_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-15T21-45-42_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-15T21-45-54_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=0.1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/neox/2022-12-14T18-27-46_neox_temp_0.1.log',
                    '../logs/neox/2022-12-15T21-45-42_neox_temp_0.1.log',
                    '../logs/neox/2022-12-15T21-45-54_neox_temp_0.1.log',
                ]

            elif TEMPERATURE == '1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-11T00-25-32_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-11T00-56-12_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/neox/2022-12-12T02-26-28_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT_NeoX(max_input_tokens=1792, temperature=1, top_p=0.9, top_k=40, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/neox/2022-12-11T00-25-32_neox_temp_1.log',
                    '../logs/neox/2022-12-11T00-56-12_neox_temp_1.log',
                    '../logs/neox/2022-12-12T02-26-28_neox_temp_1.log',
                ]

    elif MODEL == 'davinci':
        if ONE_SHOT:
            if TEMPERATURE == '0.1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/davinci/2023-01-16T00-05-06_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2023-01-16T00-05-10_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2023-01-16T00-05-13_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser, shots=True).tsv',
                ]
                LOG_PATHS = [
                    '../logs/davinci/2023-01-16T00-05-06_davinci_oneshot_temp_0.1.log',
                    '../logs/davinci/2023-01-16T00-05-10_davinci_oneshot_temp_0.1.log',
                    '../logs/davinci/2023-01-16T00-05-13_davinci_oneshot_temp_0.1.log',
                ]
        else:
            if TEMPERATURE == '0':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-14T18-30-56_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-15T16-36-13_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-15T16-36-16_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/davinci/2022-12-14T18-30-56_davinci_temp_0.log',
                    '../logs/davinci/2022-12-15T16-36-13_davinci_temp_0.log',
                    '../logs/davinci/2022-12-15T16-36-16_davinci_temp_0.log',
                ]

            elif TEMPERATURE == '0.1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-14T23-49-26_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-15T16-34-49_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-15T16-35-20_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=0.1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/davinci/2022-12-14T23-49-26_davinci_temp_0.1.log',
                    '../logs/davinci/2022-12-15T16-34-49_davinci_temp_0.1.log',
                    '../logs/davinci/2022-12-15T16-35-20_davinci_temp_0.1.log',
                ]
            
            elif TEMPERATURE == '1':
                PREDICTION_RUNS_PATHS = [
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-11-24T01-57-18_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-09T20-16-34_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                    'datasets/kleister_charity/dev-0/predictions/davinci/2022-12-12T02-27-07_LLMPipeline(prompt_variant=NeutralPrompt, model=GPT3_Davinci(max_input_tokens=3840, temperature=1, top_p=0.9, presence_penalty=0, frequency_penalty=0), parser=KleisterCharityParser).tsv',
                ]
                LOG_PATHS = [
                    '../logs/davinci/2022-11-24T01-57-18_davinci_temp_1.log',
                    '../logs/davinci/2022-12-09T20-16-34_davinci_temp_1.log',
                    '../logs/davinci/2022-12-12T02-27-07_davinci_temp_1.log',
                ]

elif SPLIT == 'test':
    pass

# Loading solution (expected.tsv)

In [ ]:
if SPLIT == 'dev':
    expected = pd.read_csv('datasets/kleister_charity/dev-0/expected.tsv', sep='\t', header=None, names=['raw'])
elif SPLIT == 'test':
    expected = pd.read_csv('datasets/kleister_charity/test-A/expected.tsv', sep='\t', header=None, names=['raw'])

for key_value_pair in expected["raw"]:
    for key_value in key_value_pair.split(" "):
        key, value = key_value.split("=")
        expected.loc[expected["raw"] == key_value_pair, key] = value

# renaming and sorting for better readability
expected.columns = ["raw", "Address (post town)", "Address (post code)", "Charity Name", "Charity Number", "Period End Date", "Address (street)", "Annual Income",  "Annual Spending"]
expected = expected[COLUMN_ORDER]

expected = expected.drop(columns=["raw"])
expected.head()

## Stats of solution

In [ ]:
print("Percentage of keys that actually have a value (are not NaN):")
expected.count() / len(expected)

# Loading predictions (three runs)

In [ ]:
prediction_runs_dfs = []
for prediction_run_path in PREDICTION_RUNS_PATHS:
    prediction_run_df = pd.read_csv(prediction_run_path, sep='\t', header=None, names=['raw'], skip_blank_lines=False)

    for raw_prediction in prediction_run_df["raw"]:
        if raw_prediction is np.nan:
            prediction_run_df.loc[prediction_run_df["raw"] == raw_prediction] = np.nan
            continue
        key_value_pairs = raw_prediction.split(" ")
        for key_value in key_value_pairs:
            key, value = key_value.split("=", 1)
            prediction_run_df.loc[prediction_run_df["raw"] == raw_prediction, key] = value

    # how many columns are there?
    num_columns = len(prediction_run_df.columns)

    column_order = ["raw", "Address (post town)", "Address (post code)", "Address (street)", "Charity Name", "Charity Number", "Period End Date", "Annual Income", "Annual Spending"]

    # rename columns
    prediction_run_df.columns = column_order[:num_columns]

    # add the missing columns and fill them with NaN
    for column in column_order[num_columns:]:
        prediction_run_df[column] = np.nan
    
    prediction_run_df = prediction_run_df[COLUMN_ORDER]
    prediction_run_df = prediction_run_df.drop(columns=["raw"])
    prediction_runs_dfs.append(prediction_run_df)

assert len(prediction_runs_dfs) == 3

## Loading associated logs of predictions (three runs)

In [ ]:
prediction_runs_logs = []
for log_path in LOG_PATHS:
    log_file = open(log_path, "r")
    log_lines = [line.strip() for line in log_file.readlines() if line.strip() and line.startswith("20") and "Raw value:" not in line]
    log_file.close()
    prediction_runs_logs.append(log_lines)

assert len(prediction_runs_logs) == 3

In [ ]:
len(prediction_runs_logs[0])

# Evaluations (answering RH1)
## RH1
> Unimodal approach can reach satisfactory performance while being more cost-efficient than current state-of-the-art multi-modal approaches

Where we define satisfactory performance as:
> 80% of the values for given and findable keys are correctly found (no distinction for the other 20%, they can be either wrong or missing (which is of course also wrong)). Correctness is defined as a case-insensitive (upper-casing everything) string match with some normalisation (details below).

## Normalisations

We normalise values for three keys as listed below. We don't normalise values of `Address (post code)`, `Charity Number`, `Period End Date`, `Annual Income`, `Annual Spending`.


### Address (post town)
Examples:
Solution | Predicted
| --- | --- |
OTTERY_ST_MARY | Ottery_St._Mary
Lichfield | City_of_Lichfield
Liverpool | City_of_Liverpool

Normalisation:
* `<Solution City>` vs. `City of <Solution City>` are both correct
* Levenshtein: edit distance of 1 (1 deletion or 1 addition, but not a substitution) counts as correct (e.g. St.Mary vs. St._Mary or St_Mary vs St._Mary)

### Address (street)
Examples: 
Solution | Predicted
| --- | --- |
36_BELLINGHAM_DRIVE | Unit_36_Bellingham_Drive
34_DECIMA_STREET | Sherborne_House,_34_Decima_Street
190_LONG_LANE | Scout_Centre,_Rear_190_Long_Lane
13_ROSSLYN_ROAD | Room_16,_ETNA_Community_Centre,_13_Rosslyn_Road
FURNIVAL_GATE | 2_Floor,_Midcity_House,_Furnival_Gate
7-14_Great_Dover_Street | 7_-_14_Great_Dover_Street
BROWNBERRIE_LANE | Leeds_Trinity_University,_Brownberrie_Lane

Normalisation: 
* Delete Spaces around "-" in both solution and prediction

Was considering generally cutting off at ,_ before or after the street but ultimately decided against it because it cannot be generally stated that having something in front or after the correct street would still make mail arrive at the destination.

Also: Levenshtein edit distance of 1 doesn't make sense here as getting the number wrong (e.g. 13 instead of 1) is a clear mistake.

### Charity Name
Examples:
Solution | Predicted
| --- | --- |
Cheltenham Ladies College | Cheltenham Ladies' College
Battersea_Dogs'_and_Cats'_Home | Battersea_Dogs'_&_Cats'_Home
Beer_Shmuel_Ltd. | Beer_Shmuel_Limited
Catch_22_Charity_Ltd. | Catch22
Richard_Hicks | Richard_Hicks_Charity
King's_Schools_Taunton_Ltd. | King's_Schools_Taunton_Limited
KEY_ENTERPRISES_(1983)_LTD. | KEY_ENTERPRISES_(1983)_LIMITED
Louth_Playgoers_Society_Ltd. | Louth_Playgoers_Society_Limited
Boxgrove_Village_Hall_and_Community_Centre | BOXGROVE_VILLAGE_HALL_&_COMMUNITY_CENTRE_CIO
London_Transport_Museum | London_Transport_Museum_Ltd.
The_Momc-Leigh_Park_Crafts_Initiative_Trust_Ltd. | THE_MOMC_-_LEIGH_PARK_CRAFTS_INITIATIVE_TRUST_LIMITED
King_Edward_Vi's_School_At_Chelmsford | King_Edward_VI_School_at_Chelmsford
The_Hope_Foundation_Ltd. | The_Hope_Foundation
Nottingham_Women's_Counselling_Service | The_Nottingham_Women's_Counselling_Service

Normalisation (+ give stats for how many values this applies):
* Cut off Ltd, Ltd. and Limited from the end of both prediction and solution 
* Replace "&" with "and" in both prediction and solution
* Delete Spaces around "-" in both prediction and solution
* Levenshtein: edit distance of 1 (1 deletion or 1 addition) counts as correct (e.g. St.Mary vs. St._Mary or St_Mary vs St._Mary, King_Edward_VI's_School vs. King_Edward_VI_School)


### Other Normalisations
Replaced uncommon character: ’ (U+2019) with ' (in the predictions and the solution) (applies to 4 values in the whole solution of the dev set)

In [ ]:
def replace_quotation_mark(df):
    """
    Replace U+2019 (right single quotation mark) with U+0027 (apostrophe) in a dataframe and return the number of replacements.
    """
    count = 0
    for column in df.columns:
        for index, value in df[column].items():
            if isinstance(value, str):
                if "’" in value:
                    df.loc[index, column] = value.replace("’", "'")
                    count += 1
    return count

for i, prediction_run_df in enumerate(prediction_runs_dfs):
    count = replace_quotation_mark(prediction_run_df)
    print(f"Replaced {count} quotation marks in prediction run {i}.")

count = replace_quotation_mark(expected)
print(f"Replaced {count} quotation marks in expected.")

In [ ]:
def is_correct(key, expected_value, predicted_value):
    """
    Our definition of correctness for each key with the normalisation rules applied.
    """
    upper_cased_expected = str(expected_value).upper()
    upper_cased_predicted = str(predicted_value).upper()

    if key == "Address (post town)":
        if upper_cased_expected.startswith("CITY_OF_") or upper_cased_expected.startswith("TOWN_OF_"):
            upper_cased_expected = upper_cased_expected[8:]
        if upper_cased_predicted.startswith("CITY_OF_") or upper_cased_predicted.startswith("TOWN_OF_"):
            upper_cased_predicted = upper_cased_predicted[8:]
        return distance(upper_cased_expected, upper_cased_predicted, weights=(1, 1, 2)) <= 1

    elif key == "Address (street)":
        upper_cased_expected = re.sub(r"(_)(-)(_)", r"\2", upper_cased_expected)
        upper_cased_predicted = re.sub(r"(_)(-)(_)", r"\2", upper_cased_predicted)
        return upper_cased_expected == upper_cased_predicted
    
    elif key == "Charity Name":
        upper_cased_expected = re.sub(r"(_LTD|_LTD.|_LIMITED)$", "", upper_cased_expected)
        upper_cased_predicted = re.sub(r"(_LTD|_LTD.|_LIMITED)$", "", upper_cased_predicted)

        upper_cased_expected = re.sub(r"(&)", "and", upper_cased_expected)
        upper_cased_predicted = re.sub(r"(&)", "and", upper_cased_predicted)

        upper_cased_expected = re.sub(r"(_)(-)(_)", r"\2", upper_cased_expected)
        upper_cased_predicted = re.sub(r"(_)(-)(_)", r"\2", upper_cased_predicted)
        return distance(upper_cased_expected, upper_cased_predicted, weights=(1, 1, 2)) <= 1
    
    else:
        return upper_cased_expected == upper_cased_predicted

## Evaluating according to own definition of "correctness" and official

In [ ]:
official_evaluations = [pd.DataFrame(np.nan, index=expected.index, columns=expected.columns) for _ in range(3)]
own_evaluations = [pd.DataFrame(np.nan, index=expected.index, columns=expected.columns) for _ in range(3)]
null_evaluations = [pd.DataFrame(np.zeros((4, len(expected.columns))), index=["TP", "FP", "FN", "TN"], columns=expected.columns) for _ in range(3)]

for i, prediction_run_df in enumerate(prediction_runs_dfs):
    for index, row in expected.iterrows():
        for column in expected.columns:
            if pd.notnull(row[column]): # because during parsing we look at the generations and if all subdocs are "null" or empty strings, it will not appear in the output and hence be NaN
                # FP: we predicted null and it was not null
                if pd.isnull(prediction_run_df.loc[index, column]):
                    null_evaluations[i].loc["FP", column] += 1

                # TN: we predicted not null (i.e. we predicted something) and it was not null
                else:
                    null_evaluations[i].loc["TN", column] += 1
                if is_correct(column, row[column], prediction_run_df.loc[index, column]):
                    own_evaluations[i].loc[index, column] = 1
                else:
                    own_evaluations[i].loc[index, column] = 0

                if str(row[column]).upper() == str(prediction_run_df.loc[index, column]).upper():
                    official_evaluations[i].loc[index, column] = 1
                else:
                    official_evaluations[i].loc[index, column] = 0

            else: # we don't care about the prediction in our own evaluation if the expected value is null
                # TP: we predicted null and it was null
                if pd.isnull(prediction_run_df.loc[index, column]):
                    null_evaluations[i].loc["TP", column] += 1
                # FN: we predicted not null and it was null
                else:
                    null_evaluations[i].loc["FN", column] += 1

                if pd.isnull(prediction_run_df.loc[index, column]):
                    official_evaluations[i].loc[index, column] = 1
                else:
                    official_evaluations[i].loc[index, column] = 0

in general we take macro average over the runs as it gives equal importance to each run (on the other hand, micro would give more importance to runs that have more predictions)

In [ ]:
# own evaluation: only looks at the keys that are actually present in the document

# combine the three runs into one by taking the mean (together with the range around the mean (e.g. if we have [1.0, 0.3, 1.7] we get 1.0 as the mean and the range is from 0.3 to 1.7)) of the own evaluations by key of each run
avg_own_evaluation_by_key = pd.concat([own_evaluation.mean(axis=0, skipna=True) for own_evaluation in own_evaluations], axis=1).agg(["mean", "min", "max", lambda x: x.max() - x.min()], axis=1)
print(f"(macro)[over runs] (own) evaluation by key:\n{avg_own_evaluation_by_key}")
print(f"(macro)[over runs and keys] (own) average of correctly predicted values: {round(avg_own_evaluation_by_key['mean'].agg('mean'), 3)}")
print(f"(macro)[over runs and keys] (own) range of correctly predicted values: {round(avg_own_evaluation_by_key['<lambda>'].agg('mean'), 3)}")

# official evaluation (same as above but with the official evaluation)
avg_official_evaluation_by_key = pd.concat([official_evaluation.mean(axis=0, skipna=True) for official_evaluation in official_evaluations], axis=1).agg(["mean", "min", "max", lambda x: x.max() - x.min()], axis=1)
print(f"(macro)[over runs] (official) evaluation by key:\n{avg_official_evaluation_by_key}")
print(f"(macro)[over runs and keys] (official) average of correctly predicted values: {round(avg_official_evaluation_by_key['mean'].agg('mean'), 3)}")
print(f"(macro)[over runs and keys] (official) range of correctly predicted values: {round(avg_official_evaluation_by_key['<lambda>'].agg('mean'), 3)}")

It is possible that the official evaluation is actually higher than our evaluation because it rewards correctly identifying null values whereas we don't care about the key-value pairs that are expected to be null.

In [ ]:
print(f"(macro)[over runs] Correlation coefficient between own and official evaluation (by key): {round(np.corrcoef(avg_own_evaluation_by_key['mean'], avg_official_evaluation_by_key['mean'])[0, 1], 3)}")

## Null Evaluation

The question we want to answer here is how good we are with predicting `null` for a key (not in subdocs, but for whole document), specifically we also look at the F1 for only `null` → `F_1_{null}`.

In [ ]:
null_scores_by_key_runs = []
for null_evaluation in null_evaluations:
    # micro average
    precision = null_evaluation.loc["TP", :].sum() / (null_evaluation.loc["TP", :].sum() + null_evaluation.loc["FP", :].sum())
    recall = null_evaluation.loc["TP", :].sum() / (null_evaluation.loc["TP", :].sum() + null_evaluation.loc["FN", :].sum())
    f1 = 2 * precision * recall / (precision + recall)
    
    # aggregating by key for later macro average
    null_scores_by_key = {}
    for key in null_evaluation.columns:   
        precision = null_evaluation.loc["TP", key] / (null_evaluation.loc["TP", key] + null_evaluation.loc["FP", key]) 
        recall = null_evaluation.loc["TP", key] / (null_evaluation.loc["TP", key] + null_evaluation.loc["FN", key])
        f1 = 2 * precision * recall / (precision + recall)
        null_scores_by_key[key] = {"precision": precision, "recall": recall, "f1": f1}

    null_scores_by_key = pd.DataFrame(null_scores_by_key).T
    null_scores_by_key_runs.append(null_scores_by_key)

print(f"(macro)[over runs] Precision for null by key:\n{pd.concat([null_scores_by_key['precision'] for null_scores_by_key in null_scores_by_key_runs], axis=1).agg(['mean', 'min', 'max'], axis=1)}")
print(f"(macro)[over runs] Recall for null by key:\n{pd.concat([null_scores_by_key['recall'] for null_scores_by_key in null_scores_by_key_runs], axis=1).agg(['mean', 'min', 'max'], axis=1)}")
print(f"(macro)[over runs] F1 score for null by key:\n{pd.concat([null_scores_by_key['f1'] for null_scores_by_key in null_scores_by_key_runs], axis=1).agg(['mean', 'min', 'max'], axis=1)}")


print(f"(macro)[over runs with micro average] Precision for null: {np.mean([null_evaluation.loc['TP', :].sum() / (null_evaluation.loc['TP', :].sum() + null_evaluation.loc['FP', :].sum()) for null_evaluation in null_evaluations])}")
print(f"(macro)[over runs with micro average] Recall for null: {np.mean([null_evaluation.loc['TP', :].sum() / (null_evaluation.loc['TP', :].sum() + null_evaluation.loc['FN', :].sum()) for null_evaluation in null_evaluations])}")
print(f"(macro)[over runs with micro average] F1 score for null: {np.mean([2 * null_evaluation.loc['TP', :].sum() / (2 * null_evaluation.loc['TP', :].sum() + null_evaluation.loc['FP', :].sum() + null_evaluation.loc['FN', :].sum()) for null_evaluation in null_evaluations])}")

print(f"(macro)[over runs with macro over keys] F1 score for null: {np.mean([null_scores_by_key['f1'].mean() for null_scores_by_key in null_scores_by_key_runs])}")

It makes sense that micro and macro average of the  are quite close together because as we saw in the beginning, almost all keys are given in the data set. There is no big "class" (key) imbalance.

## Unifications, collisions, lenient accuracy and looking at repetetiveness (all on subdocument level)

### Definition of Unification
A unification is whenever there are two or more non-null values coming from the subdoc predictions for the same key. If there is only one non-null value coming from the subdocs (which is always the case if there only is one subdoc but can also happen with more than one subdoc) then it's not a unification.

A trivial unification is a unification where all values are the same.

### Definition of Collision
Given a unification, we describe two or more different values for the same key as the unification having a collision. 

So a unification is either trivial (all values the same) or it has a collision.

### Definition of a Full Collision
Same as a collision but with the constraint that *all* values are different (not just any two).

In [ ]:
empty_key_dict = {
    "Address (post town)": None,
    "Address (post code)": None,
    "Address (street)": None,
    "Charity Name": None,
    "Charity Number": None,
    "Annual Income": None,
    "Period End Date": None,
    "Annual Spending": None,
}

runs_predictions_stats_dict = []
runs_predictions_dict = []

for i, log_lines in enumerate(prediction_runs_logs):
    print(f"Processing run {i}")
    single_run_prediction_stats_dict = []
    single_run_predictions_dict = []

    single_run_num_empty_string_predictions = 0
    for line in log_lines:
        if "Predicting document" in line: # this is the beginning of a prediction
            # create a new dictionary for this document
            single_run_prediction_stats_dict.append({
                "num_subdocs": None,
                "num_unifications": 0,
                "collision_per_key": empty_key_dict.copy(),
                "full_collision_per_key": empty_key_dict.copy(),
                "num_unified_values_per_key": empty_key_dict.copy(),
                "correct_in_any_subdoc_per_key": empty_key_dict.copy(),
                "collision_percentage": None,
                "full_collision_percentage": None,
                "correct_in_any_subdoc_percentage": None,
            })
            single_run_predictions_dict.append(empty_key_dict.copy())

        elif "Final prediction for document" in line: # this is the end of a prediction
            # calculate the percentages
            single_run_prediction_stats_dict[-1]["collision_percentage"] = np.mean([x for x in single_run_prediction_stats_dict[-1]["collision_per_key"].values() if x is not None])
            single_run_prediction_stats_dict[-1]["full_collision_percentage"] = np.mean([x for x in single_run_prediction_stats_dict[-1]["full_collision_per_key"].values() if x is not None])
            single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_percentage"] = np.mean([x for x in single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"].values() if x is not None])
    
        elif "No subdocs necessary" in line:
            single_run_prediction_stats_dict[-1]["num_subdocs"] = 1

        elif "Split document into" in line:
            num_subdocs = int(re.search("into (\d+) subdocuments", line).group(1))
            single_run_prediction_stats_dict[-1]["num_subdocs"] = num_subdocs

        elif "- parse_model_output() - Key:" in line:
            key = re.search("- parse_model_output\(\) - Key: (.*):", line).group(1)
            # the prediction is always in the next line (unless the key was not predicted at all) or the prediction is an empty string
            try:
                prediction = re.search("- parse_model_output\(\) - Stripped value: (.*)", log_lines[log_lines.index(line)+1]).group(1)
                

            except AttributeError:
                single_run_num_empty_string_predictions += 1
                prediction = "[METADATA]: EMPTY_STRING_PREDICTION"
                
            if single_run_predictions_dict[-1][key] is None:
                    single_run_predictions_dict[-1][key] = [prediction]
            else:
                single_run_predictions_dict[-1][key].append(prediction)

        elif "Unification necessary for key" in line:
            key = re.search("Unification necessary for key (.*)", line).group(1)
            single_run_prediction_stats_dict[-1]["collision_per_key"][key] = False
            single_run_prediction_stats_dict[-1]["full_collision_per_key"][key] = False
            values = re.search("Unifying \d+ \(lowered\) values (.*)", log_lines[log_lines.index(line)+1]).group(1)

            # values is the string representation of a list, so we can use eval to turn it into a list
            values = eval(values)
            unified_values = int(re.search("Unifying (\d+) \(lowered\) values", log_lines[log_lines.index(line)+1]).group(1))

            assert unified_values == len(values) # sanity check

            # if there is more than 1 value, then it's a unification (because these values don't include null values)
            if len(values) > 1:
                single_run_prediction_stats_dict[-1]["num_unifications"] += 1

                # if there are more than 1 different values, then it's a collision
                if len(set(values)) > 1:
                    single_run_prediction_stats_dict[-1]["collision_per_key"][key] = True

                # if the length of the set is equal to the length of the list, then it's a full collision
                if len(set(values)) == len(values):
                    single_run_prediction_stats_dict[-1]["full_collision_per_key"][key] = True

            
            single_run_prediction_stats_dict[-1]["num_unified_values_per_key"][key] = unified_values

            # which document are we in?
            doc_num = len(single_run_prediction_stats_dict) - 1

            # get the correct value for this key
            correct_value = expected.iloc[doc_num][key]

            # if it's NaN, then we were not supposed to predict anything for this key but we did (in at least one subdoc)
            if pd.isna(correct_value):
                single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"][key] = False # TODO: ask if this is correct
                continue

            # we have to transform the values in the list to the same format as the correct value
            values = [x.replace(" ", "_").replace(":", "_").upper() for x in values]

            # also transform the correct value to uppercase
            correct_value = str(correct_value).upper()

            # if the correct value is in the list of values, then it's correct in at least one subdoc
            if correct_value in values:
                single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"][key] = True

            else:
                single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"][key] = False

        elif "Key not found in any subdoc" in line: # null was predicted in all subdocs
            key = re.search("Key not found in any subdoc (.*)", line).group(1)
            single_run_prediction_stats_dict[-1]["collision_per_key"][key] = False
            single_run_prediction_stats_dict[-1]["full_collision_per_key"][key] = False
            single_run_prediction_stats_dict[-1]["num_unified_values_per_key"][key] = 0

            # get the correct value for this key
            correct_value = expected.iloc[len(single_run_prediction_stats_dict) - 1][key]

            if pd.isna(correct_value):
                single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"][key] = True
            else:
                single_run_prediction_stats_dict[-1]["correct_in_any_subdoc_per_key"][key] = False

    print("Number of empty string predictions:", single_run_num_empty_string_predictions)
    runs_predictions_stats_dict.append(single_run_prediction_stats_dict)
    runs_predictions_dict.append(single_run_predictions_dict)


In [ ]:
# sanity check -> all docs with x number of subdocs should have x predictions for each key (also for the keys that have an empty string prediction
# because we added [METADATA]: EMPTY_STRING_PREDICTION to the list of predictions
for _, single_run_predictions_stats_dict in enumerate(runs_predictions_stats_dict):
    for j, document in enumerate(single_run_predictions_stats_dict):
        for key in single_run_predictions_dict[j].keys():
            assert len(single_run_predictions_dict[j][key]) == single_run_prediction_stats_dict[j]["num_subdocs"]
        

In [ ]:
runs_prediction_stats_df =[pd.DataFrame(runs_predictions_stats_dict[i]) for i in range(len(runs_predictions_stats_dict))]
runs_prediction_stats_df[0].head()

In [ ]:
# replace NaN values in column `correct_in_any_subdoc_percentage` with the accuracy as per official_evaluation (this is not recorded in the log traversal)
for i in range(len(runs_prediction_stats_df)):
    runs_prediction_stats_df[i]["correct_in_any_subdoc_percentage"] = runs_prediction_stats_df[i]["correct_in_any_subdoc_percentage"].fillna(official_evaluations[i].mean(axis=1, skipna=True))

# # replace None values in column `correct_in_any_subdoc_per_key` of the rows with 1 subdoc with True/False as taken from the official_evaluation
for i, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):
    for j in range(len(single_run_prediction_stats_df)):
        if single_run_prediction_stats_df.iloc[j]["num_subdocs"] == 1:
            for key in single_run_prediction_stats_df.iloc[j]["correct_in_any_subdoc_per_key"].keys():
                single_run_prediction_stats_df.iloc[j]["correct_in_any_subdoc_per_key"][key] = bool(official_evaluations[i].iloc[j][key])

## Lenient Accuracy

Note: In the case that not in any subdoc a value was predicted (all null) we check if the correct solution is in fact null and then consider that in any subdoc the corect value (which is null) was found (in reality it was correctly identified in all). We cannot do the opposite (check if it was null in any subdoc) and then say it was correctly identified (if it was indeed null) because of the subdoc structure where any given subdoc is not guaranteed to have all key-value pairs.

In [ ]:
runs_lenient_accuracy_by_key = []

for i, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):

    # a single entry is a dictionary with the correctness for each key (True, False, or None)
    lenient_correctness_over_docs = single_run_prediction_stats_df["correct_in_any_subdoc_per_key"].tolist()

    single_run_lenient_accuracy_by_key = pd.DataFrame()

    for key in lenient_correctness_over_docs[0].keys():
        correctness = [x[key] for x in lenient_correctness_over_docs]
        avg_lenient_accuracy = np.mean([x for x in correctness if x is not None])
        single_run_lenient_accuracy_by_key[key] = [avg_lenient_accuracy]

    runs_lenient_accuracy_by_key.append(single_run_lenient_accuracy_by_key)
    print("Run", i)
    print(f"(official) (macro)[average over the keys] Lenient accuracy: {round(single_run_lenient_accuracy_by_key.mean(axis=1).mean(), 3)}")
    print(f"(official) (macro)[average over documents] Lenient accuracy: {round(single_run_prediction_stats_df['correct_in_any_subdoc_percentage'].mean(), 3)}")

print("Combined runs:")
# combine the three runs into one by taking the mean (together with the range around the mean (e.g. if we have [1.0, 0.3, 1.7] we get 1.0 as the mean and the range is from 0.3 to 1.7)) of the lenient accuracies by key of each run
avg_lenient_accuracy_by_key = pd.concat([lenient_accuracy_by_key for lenient_accuracy_by_key in runs_lenient_accuracy_by_key], axis=0).agg(["mean", "min", "max", lambda x: x.max() - x.min()], axis=0)
print(f"(official) (macro)[average over the runs] Lenient accuracy by key:\n{avg_lenient_accuracy_by_key.T}")

print(f"(official) (macro)[average over the runs and the keys] Lenient accuracy: {round(avg_lenient_accuracy_by_key.T['mean'].mean(), 3)}")
print(f"(official) (macro)[average over the runs and keys] Range of lenient accuracy by key: {round(avg_lenient_accuracy_by_key.T['<lambda>'].mean(), 3)}")

In [ ]:
runs_relative_lenient_accuracy_improvement = []
runs_absolute_lenient_accuracy_improvement = []
runs_accuracies_by_key = []

for i, official_evaluation in enumerate(official_evaluations):
    print("Run", i)
    accuracy_by_key = official_evaluation.mean(axis=0, skipna=True)
    runs_accuracies_by_key.append(accuracy_by_key)
    print("(official) Relative improvement in accuracy (lenient vs. strict):")
    for key in accuracy_by_key.keys():
        print(f"{key}: {round((runs_lenient_accuracy_by_key[i][key].values[0] - accuracy_by_key[key]) / accuracy_by_key[key] * 100, 3)}%")


    macro_relative_improvement = (single_run_lenient_accuracy_by_key.mean(axis=1).mean() - accuracy_by_key.mean()) / accuracy_by_key.mean() * 100
    print(f"(macro)[average over keys] (official) Relative improvement in accuracy (lenient vs. strict): {round(macro_relative_improvement, 3)}%")
    runs_relative_lenient_accuracy_improvement.append(macro_relative_improvement)

    macro_absolute_improvement = single_run_lenient_accuracy_by_key.mean(axis=1).mean() - accuracy_by_key.mean()
    print(f"(macro)[average over keys] (official) Absolute improvement in accuracy (lenient vs. strict): {round(macro_absolute_improvement, 3)}")
    runs_absolute_lenient_accuracy_improvement.append(macro_absolute_improvement)

print("Combined runs:")
print(f"(macro)[average over runs] (official) Relative improvement in accuracy (lenient vs. strict): {round(np.mean(runs_relative_lenient_accuracy_improvement), 3)}%")
print(f"(macro)[average over runs] (official) Absolute improvement in accuracy (lenient vs. strict): {round(np.mean(runs_absolute_lenient_accuracy_improvement), 3)}")

## Frequency and presence penalty motivation for not looking at those parameters

First, let's check how often the same value for different keys is predicted to see if it's generally correct to assume that values shouldn't be repeated (which could be combated by e.g. increasing the freq. and presence penalties).

In [ ]:
def has_two_values_for_different_keys(row):
    return len(row) == len(set(row))

num_of_same_value_rows = 0
for i in range(len(expected)):
    if not has_two_values_for_different_keys(expected.iloc[i]):
        num_of_same_value_rows += 1

print(f"Found {num_of_same_value_rows} rows that have the same value for different keys, that's {round(num_of_same_value_rows / len(expected) * 100, 3)}% of the dataset.")


So, generally speaking, there shouldn't be the same value for different keys.
A repetition of the same value would also be correct in the case of a document having more than 1 null value, where a correct generation would contain `"null"` multiple times. Let's look at how often this is the case:

In [ ]:
print("Documents that have 2 or more NaN values for any two keys:")
expected[expected.isna().sum(axis=1) >= 2]

In [ ]:
more_than_one_missing_value_in_solution_pct = round(len(expected[expected.isna().sum(axis=1) >= 2]) / len(expected) * 100, 3)
print(f"Percentage of documents that have 2 or more NaN values for any two keys: {more_than_one_missing_value_in_solution_pct}%")

So, generally speaking, we shouldn't have more than 1 null in our generations, so we can now generally assume that repetition of the same value in the generations is incorrect.

Now, let's check how often we have repetition (i.e. the same value (including null's) for different keys) in the (subdoc) predictions. If this is very high then it would make sense to look into tuning the presence and frequency penalties.

In [ ]:
for i, single_run_prediction_stats_dict in enumerate(runs_predictions_stats_dict): # for each run
    print("Run", i)
    for j in range(len(runs_predictions_stats_dict[i])): # for each document
        single_run_prediction_stats_dict[j]["pct_unique_predictions_per_subdoc"] = {} # this is min 0.125 (1/8) in the case that all predictions are the same (e.g. all 'null') and max 1.0 (all predictions are unique)
        single_run_prediction_stats_dict[j]["pct_unique_predictions_per_subdoc_ignore_null"] = {} # this is nan if all predictions are null, and also min 0.125
        for subdoc in range(single_run_prediction_stats_dict[j]["num_subdocs"]): # for each subdoc            
            # get the predictions for each key
            predictions_for_keys = [runs_predictions_dict[i][j][key][subdoc] for key in runs_predictions_dict[i][j].keys()]

            # get the number of unique predictions
            num_unique_predictions = len(set(predictions_for_keys)) # this is at least 1, cannot be 0

            # add the percentage of unique predictions to the dictionary
            single_run_prediction_stats_dict[j]["pct_unique_predictions_per_subdoc"][subdoc] = num_unique_predictions / len(predictions_for_keys)

            # get the number of unique predictions ignoring null values
            num_unique_predictions_ignore_null = len(set([x for x in predictions_for_keys if not x.upper().startswith("NULL") and x != "[METADATA]: EMPTY_STRING_PREDICTION"]))

            num_predictions_ignore_null = len([x for x in predictions_for_keys if not x.upper().startswith("NULL") and x != "[METADATA]: EMPTY_STRING_PREDICTION"]) # this is 0 if all the predictions are null or empty string

            # in this case, we set the number of unique predictions to nan
            if num_unique_predictions_ignore_null == 0:
                single_run_prediction_stats_dict[j]["pct_unique_predictions_per_subdoc_ignore_null"][subdoc] = np.nan
                continue
            # add the percentage of unique predictions (ignoring null) to the dictionary
            single_run_prediction_stats_dict[j]["pct_unique_predictions_per_subdoc_ignore_null"][subdoc] = num_unique_predictions_ignore_null / num_predictions_ignore_null

    # (micro) average percentage of unique predictions (ignoring null) per subdoc:"
    # every doc has some number of subdocs, and every subdoc has some percentage of unique predictions (ignoring null)
    # we want to average over all the subdocs in all the docs

    # first, we get the list of all the percentages of unique predictions (ignoring null) per subdoc
    all_pct_unique_predictions_per_subdoc_ignore_null = []
    for k in range(len(runs_predictions_stats_dict[i])):
        all_pct_unique_predictions_per_subdoc_ignore_null.extend(list(single_run_prediction_stats_dict[k]["pct_unique_predictions_per_subdoc_ignore_null"].values()))

    # then we average over all the percentages of unique predictions (ignoring null) per subdoc
    micro_average_pct_unique_predictions_per_subdoc_ignore_null = np.nanmean(all_pct_unique_predictions_per_subdoc_ignore_null)
    print(f"(micro) average percentage of unique predictions (ignoring null) per subdoc: {round(micro_average_pct_unique_predictions_per_subdoc_ignore_null, 3)}")

    # now we do the same thing, but for the percentage of unique predictions (not ignoring null)
    all_pct_unique_predictions_per_subdoc = []
    for k in range(len(runs_predictions_stats_dict[i])):
        all_pct_unique_predictions_per_subdoc.extend(list(single_run_prediction_stats_dict[k]["pct_unique_predictions_per_subdoc"].values()))

    micro_average_pct_unique_predictions_per_subdoc = np.nanmean(all_pct_unique_predictions_per_subdoc)
    print(f"(micro) average percentage of unique predictions per subdoc: {round(micro_average_pct_unique_predictions_per_subdoc, 3)}")

    # we do the same thing, but for the macro average (over the subdocs for each doc)
    macro_average_pct_unique_predictions_per_subdoc_ignore_null = np.nanmean([np.nanmean(list(single_run_prediction_stats_dict[k]["pct_unique_predictions_per_subdoc_ignore_null"].values())) for k in range(len(runs_predictions_stats_dict[i]))])
    print(f"(macro)[over subdocs of each doc] average percentage of unique predictions (ignoring null) per subdoc: {round(macro_average_pct_unique_predictions_per_subdoc_ignore_null, 3)}")

    macro_average_pct_unique_predictions_per_subdoc = np.nanmean([np.nanmean(list(single_run_prediction_stats_dict[k]["pct_unique_predictions_per_subdoc"].values())) for k in range(len(runs_predictions_stats_dict[i]))])
    print(f"(macro)[over subdocs of each doc] average percentage of unique predictions per subdoc: {round(macro_average_pct_unique_predictions_per_subdoc, 3)}")

print("Combined runs:")
# finally we take the macro average over the runs
# we do this by taking the average of the macro averages for each run
macro_average_pct_unique_predictions_per_subdoc_ignore_null = np.nanmean([np.nanmean([np.nanmean(list(runs_predictions_stats_dict[i][k]["pct_unique_predictions_per_subdoc_ignore_null"].values())) for k in range(len(runs_predictions_stats_dict[i]))]) for i in range(len(runs_predictions_stats_dict))])
print(f"(macro)[over runs and subdocs] average percentage of unique predictions (ignoring null) per subdoc: {round(macro_average_pct_unique_predictions_per_subdoc_ignore_null, 3)}")

# and the same but not ignoring null
macro_average_pct_unique_predictions_per_subdoc = np.nanmean([np.nanmean([np.nanmean(list(runs_predictions_stats_dict[i][k]["pct_unique_predictions_per_subdoc"].values())) for k in range(len(runs_predictions_stats_dict[i]))]) for i in range(len(runs_predictions_stats_dict))])
print(f"(macro)[over runs and subdocs] average percentage of unique predictions per subdoc: {round(macro_average_pct_unique_predictions_per_subdoc, 3)}")

It is to be expected that there is quite some repetition when not ignoring null values because many times the subdoc will only contain few or even none of the keys. When we ignore null values, repetition is almost non-existent, hence we can reasonably assume that tuning the frequency and presence penalty parameters would not improve results by much. 

## Adding number of non-null values per key

In [ ]:
for i, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df): # for each run
    single_run_prediction_stats_df.loc[:, "num_non_null_values_per_key"] = None
    for j in range(len(runs_predictions_dict[i])): # for each doc
        single_run_prediction_stats_df.at[j, "num_non_null_values_per_key"] = {key: len([x for x in runs_predictions_dict[i][j][key] if not x.upper().startswith("NULL") and x != "[METADATA]: EMPTY_STRING_PREDICTION"]) for key in runs_predictions_dict[i][j].keys()}

## Adding no_collision percentage

In [ ]:
for i, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df): # for each run
    # adding column "no_collision_percentage" which is the percentage of keys that had no collision
    single_run_prediction_stats_df.loc[:, "no_collision_percentage"] = 1 - single_run_prediction_stats_df.loc[:, "collision_percentage"]

    for j, row in single_run_prediction_stats_df.iterrows():
        num_keys = len([x for x in row["collision_per_key"].values() if x is not None])
        num_no_collisions = len([x for x in row["collision_per_key"].values() if x is False])
        try:
            single_run_prediction_stats_df.loc[j, "no_collision_percentage_calculated"] = num_no_collisions / num_keys
        except ZeroDivisionError:
            single_run_prediction_stats_df.loc[j, "no_collision_percentage_calculated"] = np.nan

    assert np.allclose(single_run_prediction_stats_df["collision_percentage"], 1 - single_run_prediction_stats_df["no_collision_percentage_calculated"], equal_nan=True) # sanity check

## Plots

In [ ]:
runs_num_of_docs_with_subdocs = []
for single_run_prediction_stats_df in runs_prediction_stats_df:
    runs_num_of_docs_with_subdocs.append(len(single_run_prediction_stats_df[single_run_prediction_stats_df["num_subdocs"] > 1]))
print(f"Number of docs with subdocs: {runs_num_of_docs_with_subdocs}")

In [ ]:
runs_num_of_docs_with_no_subdocs = []
for single_run_prediction_stats_df in runs_prediction_stats_df:
    runs_num_of_docs_with_no_subdocs.append(len(single_run_prediction_stats_df[single_run_prediction_stats_df["num_subdocs"] == 1]))
print(f"Number of docs with no subdocs: {runs_num_of_docs_with_no_subdocs}")

This is of course the same for all runs.

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

fig, ax = plt.subplots(figsize=(12, 6))

x = runs_prediction_stats_df[0]['num_subdocs'].value_counts().sort_index().index
y = runs_prediction_stats_df[0]['num_subdocs'].value_counts().sort_index().values

max_subdocs = max(x)

# add a 0 for each number of subdocs up to the maximum number of subdocs, so that there is a value for each number of subdocs
for i in range(1, max_subdocs+1):
    if i not in x:
        x = np.append(x, i)
        y = np.append(y, 0)

sns.histplot(x=x, weights=y, discrete=True, ax=ax, kde=True)

ax.set(xticks=x)
ax.set_xlim(0.5, max_subdocs)

ax.axvline(x=np.average(x, weights=y), color='red', linestyle='--', label="Mean")

# median calculated manually
z = []
for i in range(len(x)):
    z.extend([x[i]] * y[i])

ax.axvline(x=np.median(z), color='green', linestyle='--', label="Median")

ax.set(xlabel="Number of subdocuments needed")
ax.legend()
ax.set_title("Histogram of Number of Needed Subdocuments")


ax.set_xticks(np.arange(0, 51, 5))
ax.set_xlim(0.5, 51.5)

ax.set_yticks(np.arange(20, 150, 20))
ax.set_ylim(0, 145)

# plt.subplots_adjust(top=0.5)

plt.show()
fig.savefig(f"plots/{SPLIT}_{MODEL}_subdoc_hist.png", dpi=300)


In [ ]:
run_counts = []

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):
    single_run_non_null_values_per_key = {}

    for i in range(len(single_run_prediction_stats_df)):
        for key, value in single_run_prediction_stats_df.iloc[i]['num_non_null_values_per_key'].items():
            if key in single_run_non_null_values_per_key:
                single_run_non_null_values_per_key[key].append(value)
            else:
                single_run_non_null_values_per_key[key] = [value]

    fig, ax = plt.subplots(figsize=(12, 6))

    occurence_dict = {}
    for key, values in single_run_non_null_values_per_key.items():
        for value in values:
            if value in occurence_dict:
                occurence_dict[value] += 1
            else:
                occurence_dict[value] = 1

    # intersperse keys that are missing with value 0
    for i in range(0, max(occurence_dict.keys())+1):
        if i not in occurence_dict:
            occurence_dict[i] = 0

    # sort the dictionary by the key (number of non-null values)
    occurence_dict = dict(sorted(occurence_dict.items(), key=lambda item: item[0]))

    x = list(occurence_dict.keys())
    y = list(occurence_dict.values())

    run_counts.append(y)

    sns.histplot(x=x, weights=y, discrete=True, ax=ax, kde=True)

    ax.axvline(x=np.average(x, weights=y), color='red', linestyle='--', label="Mean")

    # median calculated manually
    z = []
    for i in range(len(x)):
        z.extend([x[i]] * y[i])

    ax.axvline(x=np.median(z), color='green', linestyle='--', label="Median")


    ax.set_xlim(-0.5, 24)
    ax.set_xticks(np.arange(0, 25, 1))

    ax.set_ylim(0, 3000)
    ax.set_yticks(np.arange(0, 3000, 200))

    ax.set(xlabel="Number of non-null values")
    ax.set_title("Histogram of Number of Non-Null Values per Key")
    ax.legend()

    plt.show()
    fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_RUN{run_idx}_non_null_values_vs_occurence.png", dpi=300)

In [ ]:
# combine the run_counts into one list
average_counts = []
ranges = [] # for the error bars
# the runs don't necessarily have the same length
# so we need to find the maximum length
max_length = max([len(counts) for counts in run_counts])

for i in range(max_length):
    counts = []
    for run_count in run_counts:
        if i < len(run_count):
            counts.append(run_count[i])
    average_counts.append(np.average(counts))
    ranges.append((np.min(counts), np.max(counts)))

ranges = np.array(ranges).T

# subtract the minimum from the averages (lower error)
ranges[0] = average_counts - ranges[0]

# add the maximum to the averages (upper error)
ranges[1] = ranges[1] - average_counts

fig, ax = plt.subplots(figsize=(12, 6))

x = list(range(len(average_counts)))

sns.histplot(x=x, weights=average_counts, discrete=True, ax=ax, kde=True)

# error bars
ax.errorbar(x, average_counts, yerr=ranges, fmt='none', ecolor='black', elinewidth=1, capsize=2)

# mean
ax.axvline(x=np.average(x, weights=average_counts), color='red', linestyle='--', label="Mean")

# median calculated manually
z = []
for i in range(len(x)):
    z.extend([x[i]] * round(average_counts[i]))

ax.axvline(x=np.median(z), color='green', linestyle='--', label="Median")

ax.set_xlim(-0.5, 24)
ax.set_xticks(np.arange(0, 25, 1))

ax.set_ylim(0, 3000)
ax.set_yticks(np.arange(0, 3000, 200))


ax.set(xlabel="Number of non-null values")
ax.set_title("Histogram of Number of Non-Null Values per Key")
ax.legend()
plt.show()
fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_non_null_values_vs_occurence.png", dpi=300)

In [ ]:
runs_non_null_values_by_key = []

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):

    non_null_values_by_key = {}

    for i in range(len(single_run_prediction_stats_df)):
        for key, value in single_run_prediction_stats_df.iloc[i]['num_non_null_values_per_key'].items():
            if key in non_null_values_by_key:
                non_null_values_by_key[key].append(value)
            else:
                non_null_values_by_key[key] = [value]

    fig, ax = plt.subplots(figsize=(10, 6))

    # one key = one scatter plot
    max_val = 0
    for key, values in non_null_values_by_key.items():
        occurence_dict = {}
        for value in values:
            if value in occurence_dict:
                occurence_dict[value] += 1
            else:
                occurence_dict[value] = 1

        # intersperse keys that are missing with value 0
        for i in range(0, max(occurence_dict.keys())+1):
            if i not in occurence_dict:
                occurence_dict[i] = 0

        # sort the dictionary by the key (number of unifications)
        occurence_dict = dict(sorted(occurence_dict.items(), key=lambda item: item[0]))

        x = list(occurence_dict.keys())
        y = list(occurence_dict.values())

        print(max(y))
        
        ax = sns.scatterplot(x=x, y=y, ax=ax, label=key)
        max_val = max(max_val, max(x))

    runs_non_null_values_by_key.append(non_null_values_by_key)

    ax.set(xlabel="Number of non-null values", ylabel="Count")


    ax.set_xlim(-0.5, 24.5)
    # set the x-ticks to be integers
    ax.set_xticks(np.arange(0, 25, 1))
    
    ax.set_ylim(0, 240)
    ax.set_yticks(np.arange(0, 240, 20))

    ax.legend()
    ax.set_title("Number of Non-Null Values by Key")

    plt.show()
    fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_RUN{run_idx}_non_null_values_by_key_vs_count.png", dpi=300)


In [ ]:
runs_y = []

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):

    fig, ax = plt.subplots(figsize=(8, 8))

    max_subdocs = max(single_run_prediction_stats_df['num_subdocs'])

    x = np.arange(0, max_subdocs+1)
    y = [] # number of non-null values per key (average over all keys)

    y = [[] for _ in range(max(x)+1)]

    for i in range(len(single_run_prediction_stats_df)):
        num_subdocs = single_run_prediction_stats_df.iloc[i]['num_subdocs']


        num_non_null_values = 0
        num_keys = 0
        for key, value in single_run_prediction_stats_df.iloc[i]['num_non_null_values_per_key'].items():
            num_non_null_values += value
            num_keys += 1
        if num_keys > 0:
            y[num_subdocs].append(num_non_null_values / num_keys)
        else:
            y[num_subdocs].append(0)

    y = [np.mean(y[i]) for i in range(len(y))] # average number of non-null values per key for each number of subdocuments

    runs_y.append(y)

    ax.plot([0, 52], [0, 52], color='black', linestyle='--', label='Upper bound')

    ax = sns.scatterplot(x=x, y=y, ax=ax)

    # regression line (continuing also for higher number of subdocuments than are present in the dataset)
    sns.regplot(x=x, y=y, ax=ax, scatter=False, color='red', label='Regression line', truncate=False)

    
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    ax.set_xlim(right=51.5)
    ax.set_ylim(top=51.5)

    ax.set_xticks(np.arange(5, 51, 5))
    ax.set_yticks(np.arange(5, 51, 5))

    # remove nan values from x and y
    x = [x[i] for i in range(len(x)) if not np.isnan(y[i])]
    y = [y[i] for i in range(len(y)) if not np.isnan(y[i])]


    ax.set(xlabel="Number of subdocuments", ylabel="Average number of non-null values per key")
    ax.legend(loc='upper left')
    ax.set_title("Number of Subdocuments vs. Average Number of Non-Null Values per Key")

    plt.show()
    fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_RUN{run_idx}_num_subdocs_vs_non_null_values_per_key.png", dpi=300)

    print("Correlation coefficient: " + str(round(np.corrcoef(x, y)[0, 1], 3)))


In [ ]:
# plot the average of all runs with error bars
fig, ax = plt.subplots(figsize=(8, 8))

max_subdocs = max(runs_prediction_stats_df[0]['num_subdocs'])

x = np.arange(0, max_subdocs+1)

# y is now the average over the runs in runs_y
y = [np.mean([runs_y[run_idx][i] for run_idx in range(len(runs_y))]) for i in range(len(runs_y[0]))]

# yerr is the min and max of the runs in runs_y subtracted (for min) and added (for max)
yerr = [[y[i] - min([runs_y[run_idx][i] for run_idx in range(len(runs_y))]) for i in range(len(runs_y[0]))], [max([runs_y[run_idx][i] for run_idx in range(len(runs_y))]) - y[i] for i in range(len(runs_y[0]))]]

ax.plot([0, 52], [0, 52], color='black', linestyle='--', label='Upper bound')

ax = sns.scatterplot(x=x, y=y, ax=ax)

# replace negative values in yerr with 0
yerr = [[0 if yerr[0][i] < 0 else yerr[0][i] for i in range(len(yerr[0]))], [0 if yerr[1][i] < 0 else yerr[1][i] for i in range(len(yerr[1]))]]

# error bars
ax.errorbar(x, y, yerr=yerr, fmt='none', ecolor='black', elinewidth=1, capsize=2)


sns.regplot(x=x, y=y, ax=ax, scatter=False, color='red', label='Regression line', truncate=False)

ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
ax.set_xlim(right=51.5)
ax.set_ylim(top=51.5)

ax.set_xticks(np.arange(5, 51, 5))
ax.set_yticks(np.arange(5, 51, 5))

# remove nan values from x and y
x = [x[i] for i in range(len(x)) if not np.isnan(y[i])]
y = [y[i] for i in range(len(y)) if not np.isnan(y[i])]


ax.set(xlabel="Number of subdocuments", ylabel="Average number of non-null values per key")
ax.legend(loc='upper left')
ax.set_title("Number of Subdocuments vs. Average Number of Non-Null Values per Key")

print("Correlation coefficient: " + str(round(np.corrcoef(x, y)[0, 1], 3)))


plt.show()
fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_num_subdocs_vs_non_null_values_per_key.png", dpi=300)


In [ ]:
runs_key_stats = []

for run_idx, non_null_values_by_key in enumerate(runs_non_null_values_by_key):

    x = []
    y = []

    for key, values in non_null_values_by_key.items():
        x.append(key)
        y.append(sum(values)/len(values))

    y = [round(y[i], 3) for i in range(len(y))]

    key_stats = pd.DataFrame({'key': x, 'avg_num_of_non_null_values': y})
    runs_key_stats.append(key_stats)

    print(f"RUN{run_idx}")
    print(key_stats) # avg_num_of_non_null_values is the average number of non-null values per key
    print(runs_lenient_accuracy_by_key[run_idx]) # lenient_accuracy_by_key is the accuracy for each key

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):
    fig, ax = plt.subplots(figsize=(14, 8))

    ax.axhline(y=1, color="red", linestyle="--", label="Worst case (every unification has a collision)")
    ax.axhline(y=0, color="green", linestyle="--", label="Best case (every unification is trivial)")

    single_run_prediction_stats_df.loc[single_run_prediction_stats_df['num_subdocs'] == 1, 'collision_percentage'] = 0.0
    single_run_prediction_stats_df.loc[single_run_prediction_stats_df['num_subdocs'] == 1, 'full_collision_percentage'] = 0.0
    
    ax = sns.lineplot(x="num_subdocs", y="collision_percentage", data=single_run_prediction_stats_df, ax=ax, label="Collision percentage")
    
    
    ax = sns.lineplot(x="num_subdocs", y="full_collision_percentage", data=single_run_prediction_stats_df, ax=ax, label="Full collision percentage")

    avg_collision_percentage = np.nanmean(single_run_prediction_stats_df['collision_percentage'])
    ax.axhline(y=avg_collision_percentage, color="black", linestyle="-", label="Average collision percentage")

    avg_full_collision_percentage = np.nanmean(single_run_prediction_stats_df['full_collision_percentage'])
    ax.axhline(y=avg_full_collision_percentage, color="black", linestyle="-.", label="Average full collision percentage")

    ax.set_xlim(left=0, right=51.5)
    # x-ticks from 1 to 5 with a step of 1 and then 5 to 50 with a step of 5
    ax.set_xticks(np.arange(1, 6, 1).tolist() + np.arange(5, 51, 5).tolist())

    ax.set_ylim(bottom=-0.05, top=1.05)
    ax.set_yticks(np.arange(0, 1.05, 0.1))

    ax.set(xlabel="Number of subdocuments", ylabel="Average number of collisions per key", title="Average Number of Collisions per Key vs. Number of Subdocuments")
    ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.95))

    plt.show()

    fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_RUN{run_idx}_collisions_wrt_subdocs_hist.png", dpi=300)

In [ ]:
combined_runs_prediction_stats_df = pd.concat(runs_prediction_stats_df, ignore_index=True)

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

fig, ax = plt.subplots(figsize=(14, 8))

ax.axhline(y=1, color="red", linestyle="--", label="Worst case (every unification has a collision)")
ax.axhline(y=0, color="green", linestyle="--", label="Best case (every unification is trivial)")

# add 0.0 for num_subdocs == 1
combined_runs_prediction_stats_df.loc[combined_runs_prediction_stats_df['num_subdocs'] == 1, 'collision_percentage'] = 0.0
combined_runs_prediction_stats_df.loc[combined_runs_prediction_stats_df['num_subdocs'] == 1, 'full_collision_percentage'] = 0.0
ax = sns.lineplot(x="num_subdocs", y="collision_percentage", data=combined_runs_prediction_stats_df, ax=ax, label="Collision percentage")

ax = sns.lineplot(x="num_subdocs", y="full_collision_percentage", data=combined_runs_prediction_stats_df, ax=ax, label="Full collision percentage")

avg_collision_percentage = np.nanmean(combined_runs_prediction_stats_df['collision_percentage'])
ax.axhline(y=avg_collision_percentage, color="black", linestyle="-", label="Average collision percentage")

avg_full_collision_percentage = np.nanmean(combined_runs_prediction_stats_df['full_collision_percentage'])
ax.axhline(y=avg_full_collision_percentage, color="black", linestyle="-.", label="Average full collision percentage")

ax.set_xlim(left=0, right=51.5)
# x-ticks from 1 to 5 with a step of 1 and then 5 to 50 with a step of 5
ax.set_xticks(np.arange(1, 6, 1).tolist() + np.arange(5, 51, 5).tolist())

ax.set_ylim(bottom=-0.05, top=1.05)
ax.set_yticks(np.arange(0, 1.05, 0.1))

ax.set(xlabel="Number of subdocuments", ylabel="Average number of collisions per key", title="Average Number of Collisions per Key vs. Number of Subdocuments")
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.95))

plt.show()

fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_collisions_wrt_subdocs_hist.png", dpi=300)

In [ ]:
# plot the same as above but with x_lim (right) = 10

sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

fig, ax = plt.subplots(figsize=(14, 8))

ax.axhline(y=1, color="red", linestyle="--", label="Worst case (every unification has a collision)")
ax.axhline(y=0, color="green", linestyle="--", label="Best case (every unification is trivial)")

# add 0.0 for num_subdocs == 1
combined_runs_prediction_stats_df.loc[combined_runs_prediction_stats_df['num_subdocs'] == 1, 'collision_percentage'] = 0.0
combined_runs_prediction_stats_df.loc[combined_runs_prediction_stats_df['num_subdocs'] == 1, 'full_collision_percentage'] = 0.0

ax = sns.lineplot(x="num_subdocs", y="collision_percentage", data=combined_runs_prediction_stats_df, ax=ax, label="Collision percentage")

ax = sns.lineplot(x="num_subdocs", y="full_collision_percentage", data=combined_runs_prediction_stats_df, ax=ax, label="Full collision percentage")

avg_collision_percentage = np.nanmean(combined_runs_prediction_stats_df['collision_percentage'])

ax.axhline(y=avg_collision_percentage, color="black", linestyle="-", label="Average collision percentage")

avg_full_collision_percentage = np.nanmean(combined_runs_prediction_stats_df['full_collision_percentage'])

ax.axhline(y=avg_full_collision_percentage, color="black", linestyle="-.", label="Average full collision percentage")

ax.set_xlim(left=0, right=10.5)
# x-ticks from 1 to 5 with a step of 1 and then 5 to 50 with a step of 5
ax.set_xticks(np.arange(1, 11, 1))

ax.set_ylim(bottom=-0.05, top=1.05)

ax.set_yticks(np.arange(0, 1.05, 0.1))

ax.set(xlabel="Number of subdocuments", ylabel="Average number of collisions per key", title="Average Number of Collisions per Key vs. Number of Subdocuments")
ax.legend(loc='lower right', bbox_to_anchor=(1.0, 0.04))

plt.show()

fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_collisions_wrt_subdocs_hist_zoomed.png", dpi=300)

1 subdocument == no subdocs (or in other words: the 1 subdoc is the whole document)

Note: best case assumes:
* perfect OCR
* no mistakes in the reports (no typos, no conflicting information on different pages)

of course with num_subdocs=2 every collision is also a full_collision :) 

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

runs_key_stats_trf = []
runs_avg_collision_percentage = []
runs_avg_full_collision_percentage = []

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):
    fig, ax = plt.subplots(figsize=(18, 8))

    ax.axhline(y=1, color="red", linestyle="--", label="Worst case (every unification has a collision)")

    collision_percentage_by_key = {}
    full_collision_percentage_by_key = {}


    for key in single_run_prediction_stats_df['num_unified_values_per_key'].iloc[0].keys():
        for i in range(len(single_run_prediction_stats_df)):
            if key not in collision_percentage_by_key:
                collision_percentage_by_key[key] = []

            if key not in full_collision_percentage_by_key:
                full_collision_percentage_by_key[key] = []

            collision_percentage_by_key[key].append(single_run_prediction_stats_df['collision_per_key'].iloc[i][key])
            full_collision_percentage_by_key[key].append(single_run_prediction_stats_df['full_collision_per_key'].iloc[i][key])

    # filter out None values
    collision_percentage_by_key = {key: [x for x in collision_percentage_by_key[key] if x is not None] for key in collision_percentage_by_key.keys()}
    full_collision_percentage_by_key = {key: [x for x in full_collision_percentage_by_key[key] if x is not None] for key in full_collision_percentage_by_key.keys()}

    # calculate average and ignore nan values and round to 3 decimal places
    collision_percentage_by_key = {key: round(np.nanmean(collision_percentage_by_key[key]), 3) for key in collision_percentage_by_key.keys()}
    full_collision_percentage_by_key = {key: round(np.nanmean(full_collision_percentage_by_key[key]), 3) for key in full_collision_percentage_by_key.keys()}

    # add those values to the dataframe
    runs_key_stats[run_idx]['collision_percentage'] = runs_key_stats[run_idx]['key'].map(collision_percentage_by_key)
    runs_key_stats[run_idx]['full_collision_percentage'] = runs_key_stats[run_idx]['key'].map(full_collision_percentage_by_key)

    # create a barplot that has grouped bars

    # we want to use the "hue" parameter to group the bars by collision vs. full collision
    # thus we have to transform the dataframe to have a column for each of the two types of collisions
    # and a column for the key
    key_stats_trf = runs_key_stats[run_idx].melt(id_vars=['key'], value_vars=['collision_percentage', 'full_collision_percentage'], var_name='collision_type', value_name='collision_pct')

    # rename the collision types to something more readable
    key_stats_trf['collision_type'] = key_stats_trf['collision_type'].map({'collision_percentage': 'Collision percentage', 'full_collision_percentage': 'Full Collision percentage'})

    runs_key_stats_trf.append(key_stats_trf)

    # create the barplot
    ax = sns.barplot(x="key", y="collision_pct", hue="collision_type", data=key_stats_trf, ax=ax)

    avg_collision_percentage = np.nanmean(list(collision_percentage_by_key.values()))
    ax.axhline(y=avg_collision_percentage, color="black", linestyle="-", label="Average collision percentage")
    runs_avg_collision_percentage.append(avg_collision_percentage)

    avg_full_collision_percentage = np.nanmean(list(full_collision_percentage_by_key.values()))
    ax.axhline(y=avg_full_collision_percentage, color="black", linestyle="-.", label="Average full collision percentage")
    runs_avg_full_collision_percentage.append(avg_full_collision_percentage)

    ax.set(xlabel="Key", ylabel="(Full) collision percentage", title="Collision Percentage by Key")

    ax.set_ylim(bottom=0, top=1.05)
    ax.set_yticks(np.arange(0, 1.05, 0.1))

    ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.95))

    plt.show()
    fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_RUN{run_idx}_collision_percentage_by_key.png", dpi=300)

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("colorblind")

fig, ax = plt.subplots(figsize=(18, 8))

ax.axhline(y=1, color="red", linestyle="--", label="Worst case (every unification has a collision)")

ax = sns.barplot(x="key", y="collision_pct", hue="collision_type", data=pd.concat(runs_key_stats_trf), ax=ax, errorbar=("pi", 100))
avg_collision_percentage = np.nanmean(runs_avg_collision_percentage)
ax.axhline(y=avg_collision_percentage, color="black", linestyle="-", label="Average collision percentage")
runs_avg_collision_percentage.append(avg_collision_percentage)

avg_full_collision_percentage = np.nanmean(runs_avg_full_collision_percentage)
ax.axhline(y=avg_full_collision_percentage, color="black", linestyle="-.", label="Average full collision percentage")
runs_avg_full_collision_percentage.append(avg_full_collision_percentage)

ax.set(xlabel="Key", ylabel="(Full) collision percentage", title="Collision Percentage by Key")

ax.set_ylim(bottom=0, top=1.05)
ax.set_yticks(np.arange(0, 1.05, 0.1))

ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.95))

plt.show()
fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_collision_stats_by_key_barplot.png", dpi=300)

In [ ]:
# calculate the ratio of full collision percentage to collision percentage for each run and key and attach it as a column to the dataframe
for run_idx in range(len(runs_key_stats)):
    runs_key_stats[run_idx]['full_collision_over_collision'] = round(runs_key_stats[run_idx]['full_collision_percentage'] / runs_key_stats[run_idx]['collision_percentage'], 3)

In [ ]:
combined_runs_key_stats = pd.concat(runs_key_stats)
combined_runs_lenient_accuracy_by_key = pd.concat(runs_lenient_accuracy_by_key)
combined_own_evaluation_by_key = pd.concat([own_evaluation.mean(axis=0, skipna=True) for own_evaluation in own_evaluations], axis=1)
combined_runs_accuracies_by_key = pd.concat(runs_accuracies_by_key)

In [ ]:
combined_runs_lenient_accuracy_by_key.reset_index(drop=True, inplace=True)
combined_runs_lenient_accuracy_by_key = combined_runs_lenient_accuracy_by_key.T

# put the columns below each other (instead of next to each other)
combined_runs_lenient_accuracy_by_key = combined_runs_lenient_accuracy_by_key.stack().reset_index()
combined_runs_lenient_accuracy_by_key.columns = ['key', 'run', 'lenient_accuracy']

# copy the index into a column
combined_runs_lenient_accuracy_by_key['index'] = combined_runs_lenient_accuracy_by_key.index

# change the order so that the keys of a single run are next to each other. sort by run and then by index of the row (not a real column)
combined_runs_lenient_accuracy_by_key = combined_runs_lenient_accuracy_by_key.sort_values(['run', 'index']).reset_index(drop=True)

# drop the index column
combined_runs_lenient_accuracy_by_key.drop(columns=['index'], inplace=True)


In [ ]:
# put the columns below each other (instead of next to each other)
combined_own_evaluation_by_key = combined_own_evaluation_by_key.stack().reset_index()

# rename the columns
combined_own_evaluation_by_key.columns = ['key', 'run', 'score']

# copy the index into a column
combined_own_evaluation_by_key['index'] = combined_own_evaluation_by_key.index

# change the order so that the keys of a single run are next to each other. sort by run and then by index of the row (not a real column)
combined_own_evaluation_by_key = combined_own_evaluation_by_key.sort_values(['run', 'index']).reset_index(drop=True)

# drop the index column
combined_own_evaluation_by_key.drop(columns=['index'], inplace=True)

## Some correlations

In [ ]:
print("[By key] Correlation coefficient between avg_num_of_non_null_values and collision percentage: " + str(round(np.corrcoef(combined_runs_key_stats['avg_num_of_non_null_values'], combined_runs_key_stats['collision_percentage'])[0, 1], 3)))
print("--------------------")

print("[By key] Correlation coefficient between avg_num_of_non_null_values and (own) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['avg_num_of_non_null_values'], combined_own_evaluation_by_key['score'])[0, 1], 3)))
print("[By key] Correlation coefficient between avg_num_of_non_null_values and (official) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['avg_num_of_non_null_values'], combined_runs_accuracies_by_key)[0, 1], 3)))
print("[By key] Correlation coefficient between avg_num_of_non_null_values and (official) lenient accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['avg_num_of_non_null_values'], combined_runs_lenient_accuracy_by_key['lenient_accuracy'])[0, 1], 3)))
print("--------------------")

print("[By key] Correlation coefficient between collision percentage and (own) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['collision_percentage'], combined_own_evaluation_by_key['score'])[0, 1], 3)))
print("[By key] Correlation coefficient between collision percentage and (official) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['collision_percentage'], combined_runs_accuracies_by_key)[0, 1], 3)))
print("[By key] Correlation coefficient between collision percentage and (official) lenient accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['collision_percentage'], combined_runs_lenient_accuracy_by_key['lenient_accuracy'])[0, 1], 3)))
print("--------------------")

print("[By key] Correlation coefficient between full collision percentage and (own) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['full_collision_percentage'], combined_own_evaluation_by_key['score'])[0, 1], 3)))
print("[By key] Correlation coefficient between full collision percentage and (official) accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['full_collision_percentage'], combined_runs_accuracies_by_key)[0, 1], 3)))
print("[By key] Correlation coefficient between full collision percentage and (official) lenient accuracy: " + str(round(np.corrcoef(combined_runs_key_stats['full_collision_percentage'], combined_runs_lenient_accuracy_by_key['lenient_accuracy'])[0, 1], 3)))
print("--------------------")

In [ ]:
runs_key_stats[0]

In [ ]:
combined_runs_key_stats

In [ ]:
# group the rows by key and leave the order of the rows intact
combined_runs_key_stats_grouped = combined_runs_key_stats.groupby('key', as_index=False).first()

# re-calculate full_collision_over_collision by using the new collision_percentage and full_collision_percentage
combined_runs_key_stats_grouped['full_collision_over_collision'] = combined_runs_key_stats_grouped['full_collision_percentage'] / combined_runs_key_stats_grouped['collision_percentage']

# reorder the rows
# COLUMN_ORDER = ["raw", "Address (post town)", "Address (post code)", "Address (street)", "Charity Name", "Charity Number", "Annual Income", "Period End Date", "Annual Spending"]
combined_runs_key_stats_grouped = combined_runs_key_stats_grouped.reindex([1, 0, 2, 5, 6, 3, 7, 4]).reset_index(drop=True)


In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))

# draw a heatmap and fix the colorbar from 0 to 1
sns.heatmap(combined_runs_key_stats_grouped.iloc[:, 2:], annot=True, fmt=".2f", cmap="YlGnBu", ax=ax, vmin=0, vmax=1)
ax.set(xlabel="Metric", ylabel="Key", title="Collision Statistics by Key")

ax.set_xticklabels(['Collision Percentage', 'Full Collision Percentage', 'Full Collision over Collision'])

# set the yticks to the key names
ax.set_yticklabels(combined_runs_key_stats_grouped['key'], rotation=0)

plt.show()

fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_collision_stats_by_key.png", dpi=300)

In [ ]:
# attach the own_evaluation.mean(axis=1, skipna=True) to the prediction_stats dataframe
for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):
    single_run_prediction_stats_df['own_evaluation_accuracy'] = own_evaluations[run_idx].mean(axis=1, skipna=True)

runs_prediction_stats_df[0].head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

max_subdocs = max(runs_prediction_stats_df[0]['num_subdocs'])

x = np.arange(0, max_subdocs+1)
runs_y = []

for run_idx, single_run_prediction_stats_df in enumerate(runs_prediction_stats_df):

    single_run_y = [] # avg. accuracy for each number of subdocs
    # yerr = [] # standard deviation for each number of subdocs

    # scatter plot of the avg. own_evaluation accuracy vs num_subdocs
    for i in range(0, max_subdocs+1):
        single_run_y.append(runs_prediction_stats_df[run_idx][runs_prediction_stats_df[run_idx]['num_subdocs'] == i]['own_evaluation_accuracy'].mean())
        #yerr.append(prediction_stats[prediction_stats['num_subdocs'] == i]['own_evaluation_accuracy'].std())

    runs_y.append(single_run_y)


# combined y is the average of the runs_y
y = np.mean(runs_y, axis=0)

# yerr is the min and max of the runs in runs_y subtracted (for min) and added (for max)
yerr = [[y[i] - min([runs_y[run_idx][i] for run_idx in range(len(runs_y))]) for i in range(len(runs_y[0]))], [max([runs_y[run_idx][i] for run_idx in range(len(runs_y))]) - y[i] for i in range(len(runs_y[0]))]]

# replace very small negative values (due to rounding errors) with 0
yerr = [[0 if yerr[0][i] > -0.0001 else yerr[0][i] for i in range(len(yerr[0]))], [0 if yerr[1][i] < 0.0001 else yerr[1][i] for i in range(len(yerr[1]))]]

ax = sns.scatterplot(x=x, y=y, ax=ax)

# plot the error bars
ax.errorbar(x, y, yerr=yerr, fmt='none', ecolor='black', elinewidth=1, capsize=2)

# remove nan values from x and y
x = [x[i] for i in range(len(x)) if not np.isnan(y[i])]
y = [y[i] for i in range(len(y)) if not np.isnan(y[i])]

mpl.rcParams['axes.xmargin'] = 1  # set very wide margins: 100% of the x-axis range so that the regression line is not cut off

# regression line
sns.regplot(x=x, y=y, ax=ax, scatter=False, color='red', label='Regression line', truncate=False)

# set the labels
ax.set(xlabel="Number of subdocuments", ylabel="Accuracy", title="Accuracy by number of subdocuments")

ax.set_xlim(0, 52)
ax.set_xticks(np.arange(0, 52, 5))

ax.set_ylim(0, 0.75)
ax.set_yticks(np.arange(0, 0.76, 0.05))



ax.legend(loc='upper right')
ax.set_title("Number of Subdocuments vs. Accuracy (own evaluation)")

plt.show()
fig.savefig(f"plots/{SPLIT}_{MODEL}_{TEMPERATURE}_average_own_eval_accuracy_vs_num_subdocs.png", dpi=300)

print("Correlation coefficient: " + str(round(np.corrcoef(x, y)[0, 1], 3)))